# Calculating Spike Rates From Lay Files

MNE has some useful features that will allow us to quickly read the spike data and calculate statistics of interest.

**Note**: mne must have version 0.23 or else there is a bug in the annotation reader

In [1]:
from pathlib import Path
import mne

### First, we can read in the .lay file into mne's raw object

In [9]:
lay_fpath = "D:/OneDriveParent/OneDrive - Johns Hopkins/spike_detection/data/3-archive.lay"  # Change me
raw = mne.io.read_raw_persyst(lay_fpath)
verbose = True

Loading D:\OneDriveParent\OneDrive - Johns Hopkins\spike_detection\data\3-archive.lay


### We can then extract important information from this object

Here, we get the annotations, duration of the recording, and number of total channels

In [10]:
annotations = raw.annotations
mins = raw.n_times/(60*raw.info['sfreq'])  # We will be calculating spikes/min in each channel
n_chs = raw.info['nchan']
if verbose:
    print(f"mins: {mins}; n_chs: {n_chs}")
    [print(a['description']) for a in annotations]

mins: 16.35; n_chs: 43
+0.000000
segment: rec start longit eeg
a1+a2 off
eyes closed
@warning: persyst is post processing. detection notifications are turned off.
+1.000000
10-11 hz
+4.000000
eyes open
eyes closed
+12.000000
eyes open
+18.000000
eyes closed
+24.000000
eyes open
+33.000000
eyes closed
+42.000000
+55.000000
ask date
+60.000000
answer yes
+64.000000
ask hosp
answer yes
+66.000000
+72.000000
ask dob
answer yes
+76.000000
+81.000000
ask age
answer yes
+85.000000
+92.000000
ask 100-7=
answer yes
+96.000000
+100.000000
ask 93-7=
answer yes
+110.000000
+140.000000
photo 1hz
+145.000000
photo 3hz
+150.000000
photo 5hz
+155.000000
photo 7hz
+160.000000
photo 9hz
+165.000000
photo 11hz
+171.000000
* gen swc frag
photo 13hz
z ant predom/bifront
z slightly r max
+172.000000
no clinical
polyspike admixed
spike f4-av12 0.90
+176.000000
photo 15hz
+180.000000
photo 17hz
+185.000000
photo 19hz
+190.000000
photo 21hz
+195.000000
photo 20hz
+200.000000
photo 18hz
+206.000000
photo 16hz
+

### We now make sure we are only extracting the Spike annotations

We can also get rid of all the information except for the channel name

In [11]:
spike_channels = [a['description'].split(' ')[1] for a in annotations if 'spike' in a['description']]
if verbose:
    print(spike_channels)

['admixed', 'f4-av12', 'fp2-av12', 't3-av12', 'f7-av12', 'f7-av12', 'o1-av12', 'fp2-av12', 'fp2-av12', 't5-av12', 'fp1-av12', 'fp1-av12', 'fp2-av12', 'some', 'f7-av12', 'f4-av12', 'fp2-av12', 'fp1-av12', 't5-av12', 't5-av12', 't6-av12', 'p3-av12', 't5-av12', 'f8-av12']


### And now we can extract relevant rates

In [5]:
def count_spikes(spike_list):
    spike_dict = {}
    for ch in list(set(spike_list)):
        spike_dict.update({ch: spike_list.count(ch)})
    return spike_dict

In [6]:
spike_counts = count_spikes(spike_channels)
spike_rates = {}
for key, val in spike_counts.items():
    spike_rates[key] = val / mins
total_spike_rate = sum(spike_counts.values()) / (n_chs*mins)  # normalizing by total time and number of channels
if verbose:
    print("Spike Counts:")
    [print(f"{key}: {value}") for key, value in spike_counts.items()]
    print("\n\nSpike Rates:")
    [print(f"{key}: {value}") for key, value in spike_rates.items()]
    print(f"\n\nTotal Spike Rate: {total_spike_rate} spikes/min*ch")

Spike Counts:
p3-av12: 1
f3-av12: 1
p4-av12: 1
t3-av12: 6
t5-av12: 2
cz-av12: 4
f7-av12: 14
pz-fp12: 1


Spike Rates:
p3-av12: 0.05946481665014866
f3-av12: 0.05946481665014866
p4-av12: 0.05946481665014866
t3-av12: 0.35678889990089196
t5-av12: 0.11892963330029732
cz-av12: 0.23785926660059464
f7-av12: 0.8325074331020813
pz-fp12: 0.05946481665014866


Total Spike Rate: 0.04148708138382465 spikes/min*ch
